In [1]:
import pandas as pd
import numpy as np
import glob

In [3]:
masses = [0.18687, 0.20264, 0.24948, 0.30559,
          0.33327, 0.35153, 0.38445, 0.39855,
          0.41599, 0.44810]
path = lambda x: 'he-SDSS/m{}He.SDSS'.format(x)
path(masses[0])

'he-SDSS/m0.18687He.SDSS'